有些版是18+的，所以要多一些判定。  
normal 200頁 40min  
18+    200頁 1h30min

In [1]:
import warnings
warnings.filterwarnings("ignore")

# for normal

In [12]:
import csv
import numpy as np
import re
import requests
from bs4 import BeautifulSoup

cur_page = 'https://www.ptt.cc/bbs/movie/index8671.html'
next_page =''

with open('movie_max.csv', 'w', errors='ignore',newline='') as csvfile:
    
    writer = csv.writer(csvfile)
    
    for i in range(1,201):        #this to control num of pages
        article_url = []
        article_vote = []
        article_context = []
        article_time = []
        article_title=[]
        false_index = []
        table = []
    
        res = requests.get(cur_page, verify=False)
        soup = BeautifulSoup(res.text)
        res.close()
    
        next_page = 'https://www.ptt.cc'+soup.select('.action-bar')[0].find_all('a')[3]['href']
        for entry in soup.select('.r-ent'):
            if not entry.find('a'):
                continue
            article_vote.append(entry.select('.nrec')[0].text)
            article_title.append(entry.select('.title')[0].text)
            article_url.append('https://www.ptt.cc'+entry.find('a')['href'])
        
 
        for i,page in enumerate(article_url):
            res_loop = requests.get(page, verify=False)
            soup_loop = BeautifulSoup(res_loop.text)
            res_loop.close()
            
            if (len(soup_loop.select('.article-meta-value'))!=4):
                false_index.append(i)
                continue
            if not soup_loop.find(id="main-content"):
                false_index.append(i)
                continue    
                
            content = soup_loop.find(id="main-content").text
            delete_content_1 = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
            delete_content_2 = soup_loop.select('.article-meta-value')[3].text
            content = content.split(delete_content_1)
            content = content[0].split(delete_content_2)
            main_content = content[1].replace('--', '')     
            
            main_content = re.sub('\n','\n\n',main_content)
            main_content = re.sub('\n[:※]+[^\n:※]+\n','',main_content)
            main_content = re.sub('\n\n','\n',main_content)
            
            article_context.append(main_content)
            article_time.append(delete_content_2)
        false_index.reverse()
        for i in false_index:
            del article_vote[i]
            del article_title[i]
            del article_url[i]
    
        table = [article_url,article_vote,article_context,article_time,article_title]
        table = np.transpose(table).tolist()
        writer.writerows(table)
    
    
        cur_page = next_page

# for 18+

In [14]:
import csv
import numpy as np
import re

cur_page = 'https://www.ptt.cc/bbs/sex/index4005.html'
next_page =''

with open('sex_max.csv', 'w', errors='ignore',newline='') as csvfile:
    
    writer = csv.writer(csvfile)
    
    for i in range(1,201):        #this to control num of pages
        article_url = []
        article_vote = []
        article_context = []
        article_time = []
        article_title=[]
        false_index = []
        table = []
        
        payload = {'from': cur_page, 'yes':'yes'}
        rs = requests.session()
        res = rs.post('https://www.ptt.cc/ask/over18', verify=False, data=payload)
        res = rs.get(cur_page, verify=False)
        soup = BeautifulSoup(res.text)
        rs.close()
    
        next_page = 'https://www.ptt.cc'+soup.select('.action-bar')[0].find_all('a')[3]['href']
        for entry in soup.select('.r-ent'):
            if not entry.find('a'):
                continue
            article_vote.append(entry.select('.nrec')[0].text)
            article_title.append(entry.select('.title')[0].text)
            article_url.append('https://www.ptt.cc'+entry.find('a')['href'])
        
 
        for i,page in enumerate(article_url):
        
            payload = {'from': page, 'yes':'yes'}
            rs = requests.session()
            res = rs.post('https://www.ptt.cc/ask/over18', verify=False, data=payload)
            res_loop = rs.get(page, verify=False)
            soup_loop = BeautifulSoup(res_loop.text)
            rs.close()
            
            if (len(soup_loop.select('.article-meta-value'))!=4):
                false_index.append(i)
                continue
            if not soup_loop.find(id="main-content"):
                false_index.append(i)
                continue
            
            content = soup_loop.find(id="main-content").text
            delete_content_1 = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
            delete_content_2 = soup_loop.select('.article-meta-value')[3].text
            content = content.split(delete_content_1)
            content = content[0].split(delete_content_2)
            main_content = content[1].replace('--', '')     
            
            main_content = re.sub('\n','\n\n',main_content)
            main_content = re.sub('\n[:※]+[^\n:※]+\n','',main_content)
            main_content = re.sub('\n\n','\n',main_content)
            
            article_context.append(main_content)
            article_time.append(delete_content_2)
        false_index.reverse()
        for i in false_index:
            del article_vote[i]
            del article_title[i]
            del article_url[i]
    
        table = [article_url,article_vote,article_context,article_time,article_title]
        table = np.transpose(table).tolist()
        writer.writerows(table)
    
    
        cur_page = next_page